In [26]:
import copy
import cv2 as cv
import math
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import pytesseract
import os

In [36]:
def show_wait_destroy(winname, img):
    cv.imshow(winname, img)
    cv.moveWindow(winname, 500, 0)
    cv.waitKey(0)
    cv.destroyWindow(winname)

In [177]:
image = cv.imread("imgs/crop.jpg",cv.IMREAD_COLOR)
show_wait_destroy("image",image)
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
show_wait_destroy("gray", gray)
gray = cv.bitwise_not(gray)
bw = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 15, -2)
show_wait_destroy("binary", bw)
# enlarge = cv2.resize(image_copy,None,fx=2.0, fy=2.0, interpolation = cv2.INTER_CUBIC)
# cv2.imshow("image",enlarge)
# cv2.waitKey(0)
# denoiced_image = cv2.fastNlMeansDenoisingColored(image_copy,None,10,10,7,21)
# cv2.imshow("denoiced_image",denoiced_image)
# cv2.waitKey(0)


In [178]:
horizontal = np.copy(bw)
vertical = np.copy(bw)
cols = horizontal.shape[1]
horizontal_size = int(cols / 5)
horizontalStructure = cv.getStructuringElement(cv.MORPH_RECT, (horizontal_size, 1))
# Apply morphology operations
horizontal = cv.erode(horizontal, horizontalStructure)
horizontal = cv.dilate(horizontal, horizontalStructure)

# Show extracted horizontal lines
show_wait_destroy("horizontal", horizontal)

In [179]:
rows = vertical.shape[0]
verticalsize = int(rows / 5)
# Create structure element for extracting vertical lines through morphology operations
verticalStructure = cv.getStructuringElement(cv.MORPH_RECT, (1, verticalsize))

# Apply morphology operations
vertical = cv.erode(vertical, verticalStructure)
vertical = cv.dilate(vertical, verticalStructure)

# Show extracted vertical lines
show_wait_destroy("vertical", vertical)

In [180]:
points = cv.bitwise_and(horizontal,vertical)
show_wait_destroy("Points of Intersection", points)

In [182]:
table = horizontal + vertical
show_wait_destroy("table",table)

In [79]:
print(points.shape)

(218, 857)


In [183]:
x = 0
y = 0
co_ordinates = []
temp = []
while x < (points.shape[0]):
    while y < (points.shape[1]):
        if points[x,y] == 255:
            temp.append((x,y))
            y = y+5
        else:
            y = y+1
    
    if len(temp) != 0:
        co_ordinates.append(temp)
    x = x+1
    y = 0    
    temp = []        

In [84]:
print(co_ordinates)

[[(5, 9), (5, 38)], [(55, 8), (55, 38)], [(105, 8), (105, 37)], [(155, 8), (155, 37)], [(205, 8), (205, 37)]]


In [189]:
x = 0
y = 0
new_image = np.copy(image)
while x < table.shape[0]:
    while y < table.shape[1]:
        if table[x,y] == 255:
            new_image[x,y,0] = 0
            new_image[x,y,1] = 0
            new_image[x,y,2] = 0
        y = y+1
    x = x+1
    y = 0
show_wait_destroy("newImage", new_image)

In [152]:
text = pytesseract.image_to_string(new_image)

In [ ]:
himage = np.copy(table)
line_image = np.copy(new_image)
# gray = cv.cvtColor(himage, cv.COLOR_BGR2GRAY)
# edges = cv.Canny(gray, 50, 150, apertureSize=3)

lines = cv.HoughLinesP(himage,1,np.pi/180,20,20)[0].tolist()
lines = remove_duplicates(lines)
for x1, y1, x2, y2 in lines:
        cv.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 1)
# for line in lines:
#     coord = line[0]
#     cv.line(line_image, (coord[0],coord[1]), (coord[2],coord[3]), [0,0,255], 2)
#     show_wait_destroy("newImage", line_image)
    
show_wait_destroy("newImage", line_image)

In [194]:
def remove_duplicates(lines):
    # remove duplicate lines (lines within 10 pixels of eachother)
    for x1, y1, x2, y2 in lines:
        for index, (x3, y3, x4, y4) in enumerate(lines):
            if y1 == y2 and y3 == y4:
                diff = abs(y1-y3)
            elif x1 == x2 and x3 == x4:
                diff = abs(x1-x3)
            else:
                diff = 0
            if diff < 10 and diff is not 0:
                del lines[index]
    return lines

In [54]:
# edges = cv.adaptiveThreshold(vertical, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 3, -2)
# show_wait_destroy("edges", edges)


# kernel = np.ones((2, 2), np.uint8)
# edges = cv.dilate(edges, kernel)
# show_wait_destroy("dilate", edges)

# smooth = np.copy(vertical)

# smooth = cv.blur(smooth, (2, 2))

# (rows, cols) = np.where(edges != 0)
# vertical[rows, cols] = smooth[rows, cols]

# show_wait_destroy("smooth - final", vertical)
# If we consider (0,0) as top left corner of image called im with 
# left-to-right as x direction and top-to-bottom as y direction. and we have (x1,y1) 
# as the top-left vertex and (x2,y2) as the bottom-right vertex of a rectangle region within that image, then:

# roi = im[y1:y2, x1:x2]